In [ ]:
import pandas as pd

train = pd.read_csv("/content/drive/MyDrive/CODECLONE/train.csv")
test = pd.read_csv("/content/drive/MyDrive/CODECLONE/test.csv")
valid = pd.read_csv("/content/drive/MyDrive/CODECLONE/valid.csv")

In [ ]:
import re
def preprocess_java_code(code):
    # 1. Remove single-line and multi-line comments
    code = re.sub(r'//.*?\n|/\*.*?\*/', '', code, flags=re.S)

    # 2. Remove string literals
    code = re.sub(r'"(?:\\.|[^"\\])*"|\'(?:\\.|[^\'\\])*\'', 'STRING_LITERAL', code)

    # 3. Remove numeric literals
    code = re.sub(r'\b\d+(\.\d+)?\b', 'NUMERIC_LITERAL', code)

    # 4. Normalize case
    code = code.lower()

    # 5. Tokenize the code
    tokens = re.findall(r'\w+|[^\w\s]', code)

    # 6. Remove unnecessary whitespace
    processed_code = ' '.join(tokens)

    return processed_code

In [ ]:
train['function1'] = train['func_x'].apply(preprocess_java_code)
train['function2'] = train['func_y'].apply(preprocess_java_code)

test['function1'] = test['func_x'].apply(preprocess_java_code)
test['function2'] = test['func_y'].apply(preprocess_java_code)

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

def embed_code(code, tokenizer, model):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    tokens = tokenizer.encode(code, return_tensors='pt', truncation=True, max_length=512)
    tokens = tokens.to(device)
    embeddings = model(tokens)[0].mean(dim=1).detach().cpu().numpy()
    return embeddings


In [ ]:
# Tokenize and embed using CodeBERT
tokenizer = AutoTokenizer.from_pretrained('microsoft/codebert-base')
model = AutoModel.from_pretrained('microsoft/codebert-base')


train['function1_embedded'] = train['function1'].apply(lambda x: embed_code(x, tokenizer, model))
train['function2_embedded'] = train['function2'].apply(lambda x: embed_code(x, tokenizer, model))
test['function1_embedded'] = test['function1'].apply(lambda x: embed_code(x, tokenizer, model))
test['function2_embedded'] = test['function2'].apply(lambda x: embed_code(x, tokenizer, model))

In [ ]:
X_train = np.hstack((np.vstack(train['function1_embedded'].values), np.vstack(train['function2_embedded'].values)))
X_test = np.hstack((np.vstack(test['function1_embedded'].values), np.vstack(test['function2_embedded'].values)))

Y_train = train['Label'].values
Y_test = test['Label'].values

In [ ]:
# Train an ML model
clf = RandomForestClassifier(random_state=42)  # or clf = SVC(kernel='linear', C=1, random_state=42)
clf.fit(X_train, Y_train)

# Evaluate the model
y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(Y_test, y_pred))
print("Accuracy: ", accuracy_score(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.95      0.96      1481
           1       0.68      0.84      0.76       209

    accuracy                           0.93      1690
   macro avg       0.83      0.89      0.86      1690
weighted avg       0.94      0.93      0.94      1690

Accuracy:  0.9325443786982248
